In [3]:
import math
import numpy
import random

In [14]:
class Parameters:
    MAX_NODES = math.pow(2,20)

class Gene:
    innovation_number:int = 0
    def __init__(self,innovation_number):
        self.innovation_number = innovation_number

class NodeGene(Gene):
    # Allow connections from low x value to high y value
    x = 0
    y = 0
    def __init__(self,innovation_number):
        super.__init__(innovation_number)
        self.x = 0
        self.y = 0

    def equals(self, object):
        if not type(object) is Gene:
            return False
        
        return self.innovation_number == object.innovation_number

    def __str__(self):
        return f"(NodeGene: {self.innovation_number})"

    def hashCode(self):
        return self.innovation_number
    
class ConnectionGene(Gene):
    from_gene:NodeGene = None
    to_gene:NodeGene = None

    weight:float = 0
    enabled:bool = True

    replace_index:int = 0

    def __init__(self,from_gene:NodeGene,to_gene:NodeGene):
        self.from_gene = from_gene
        self.to_gene = to_gene

    def equals(self, object):
        if not type(object) is ConnectionGene:
            return False
        
        return self.from_gene.equals(object.from_gene) and self.to_gene.equals(object.to_gene)

    def __str__(self):
        return f"(ConnectionGene: from={self.from_gene.innovation_number},to={self.to_gene.innovation_number},weight={self.weight},enabled={self.enabled},innovation_number={self.innovation_number})"

    def hashCode(self):
        return self.from_gene.innovation_number * Parameters.MAX_NODES + self.to_gene.innovation_number
    

In [15]:
# data structures
class RandomHashSet:
    hashSet = set() # <T>
    data = []  # <T>

    def __init__(self):
        self.hashSet = set()
        self.data = []

    def contains(self, object)->bool:
        return object in self.hashSet

    def randomElement(self):
        len_hash = self.size()
        if len_hash > 0:
            return self.data[random.randint(0,len_hash-1)]
        return None

    def size(self):
        return len(self.data)

    def add(self,object):
        if object not in self.hashSet:
            self.hashSet.add(object)
            self.data.append(object)
    
    def addSorted(self,object:Gene):

        for i in range(self.size()):
            gene:Gene = self.data[i]
            innovation = gene.innovation_number
            if object.innovation_number < innovation:
                self.hashSet.add(object)
                self.data.insert(i,object)
                return


        self.hashSet.add(object)
        self.data.append(object)

    def clear(self):
        self.hashSet.clear()
        self.data.clear()

    def get(self,index:int):
        if index < 0 or index >= self.size():
            return None
        return self.data[index]
        
    def removeByIndex(self,index:int):
        if index < 0 or index >= self.size():
            return None
        self.hashSet.remove(self.data[index])
        self.data.pop(index)
        
    def remove(self,object):
        self.hashSet.remove(object)
        self.data.remove(object)


class RandomSelector:

    objects = []  # <T>
    scores = [] # double

    total_score = 0 # double

    def __init__(self):

        self.objects = []
        self.scores = [] # double
        self.total_score = 0 # double

    def add(self,element, score:float):
        self.objects.append(element)
        self.scores.append(score)
        self.total_score += score

    def random(self):
        v: float = random.random() * self.total_score
        c: float = 0
        for i in rande(len(self.objects)):
            c += self.scores[i]
            if c >= v:
                return self.objects[i]
        return None

    def reset(self):
        self.objects.clear()
        self.scores.clear()
        self.total_score = 0



In [17]:
class Node:
    # crossed reference with Connection
    x:float = 0
    output:float = 0
    connections:list = []  # type: Connection

    def __init__(self,x:float):
        self.x = x

    def calculate(self):
        s:float = 0
        for i in range(len(self.connections)):
            c:Connection = self.connections[i]
            if c.enabled:
                s += c.weight * c.from_node.output

        self.output = self.activationFunction(s)

    def activationFunction(self, x:float)->float:
        return float(1) / (1 + math.exp(-x))

    def compareTo(self, o):
        # o: Node
        if self.x > o.x:
            return -1
        if self.x < o.x:
            return 1
        return 0

class Connection:
    # crossed reference with Node
    from_node:Node = None
    to_node:Node = None

    weight:float = 0
    enabled:bool = True

    def __init__(self,from_node:Node,to_node:Node):
        self.from_node = from_node
        self.to_node = to_node


class Calculator:
    input_nodes:list = [] # type: Node
    hidden_nodes:list = [] # type: Node
    output_nodes:list = [] # type: Node

    def __init__(self,nodes:RandomHashSet,cons:RandomHashSet):
        # nodes:list[NodeGene] <- g.nodes:list[ConnectionGene], cons <- g.connections

        nodeHashMap = dict() # HashMap<Integer, Node>
        for i in range(len(nodes)):
            n:NodeGene = nodes[i]
            node:Node = Node(n.x)

            nodeHashMap[n.innovation_number] = node

            if n.x <= 0.1:
                self.input_nodes.append(node)
            elif n.x >= 0.9:
                self.output_nodes.append(node)
            else:
                self.hidden_nodes.append(node)
        
        self.hidden_nodes.sort(key=lambda nod: nod.x)

        for i in range(len(cons.data)):
            c:ConnectionGene = cons.data[i]
            from_gene:NodeGene = c.from_gene
            to_gene:NodeGene = c.to_gene

            from_node:Node = nodeHashMap[from_gene.innovation_number]
            to_node:Node = nodeHashMap[to_gene.innovation_number]

            con:Connection = Connection(from_node,to_node)
            con.weight = c.weight
            con.enabled = c.enabled

            to_node.connections.append(con)


    def calculate(self, input:list)->list:
        # return float[]
        if len(input) != len(self.input_nodes):
            raise Exception("Data doesn't fit")
        
        for i in range(len(self.input_nodes)):
            self.input_nodes[i].output = input[i]

        for n in self.hidden_nodes:
            n.calculate()

        output:list = [0 for _ in range(len(self.output_nodes))]

        for i in range(len(self.output_nodes)):
            self.output_nodes[i].calculate()
            output[i] = self.output_nodes[i].output

        return output


class Client:
    calculator:Calculator = None

class Species:
    pass

class Neat:
    C1:float = 1
    C2:float = 1
    C3:float = 1
    CP:float = 4

    WEIGHT_SHIFT_STRENGTH:float = 0.3
    WEIGHT_RANDOM_STRENGTH:float = 1

    SURVIVORS:float = 0.8

    PROBABILITY_MUTATE_LINK:float = 0.01
    PROBABILITY_MUTATE_NODE:float = 0.1
    PROBABILITY_MUTATE_WEIGHT_SHIFT:float = 0.02
    PROBABILITY_MUTATE_WEIGHT_RANDOM:float = 0.02
    PROBABILITY_MUTATE_TOGGLE_LINK:float = 0

    all_connections = dict() # type hashmap<ConnectionGene, ConnectionGene>

    all_nodes:RandomHashSet = None # type NodeGene
    clients:RandomHashSet = None # type Client
    species:RandomHashSet = None # type Species

    max_clients:int = 0
    output_size:int = 0
    input_size:int = 0



    def __init__(self,input_size:int, output_size:int, clients:int):

        self.all_nodes = RandomHashSet()
        self.clients = RandomHashSet()
        self.species = RandomHashSet()

        self.reset(input_size, output_size, clients)


    def reset(self, input_size:int, output_size:int, clients:int)->None:
        self.input_size = input_size
        self.output_size = output_size
        self.clients = clients

        self.all_connections.clear()
        self.all_nodes.clear()
        self.clients.clear()

        for i in range(input_size):
            n:NodeGene = self.getNode()
            n.x = 0.1
            n.y = (i+1)/float(input_size+1)

        for i in range(output_size):
            n:NodeGene = self.getNode()
            n.x = 0.9
            n.y = (i+1)/float(output_size+1)

        for i in range(self.max_clients):
            c:Client = Client()
            c.genome = Genome.empty_genome(self)
            c.generateCalculator()
            self.clients.add(c)




class Genome:
    connections:RandomHashSet = None  # ConnectionGene type
    nodes:RandomHashSet = None # NodeGene type
    neat:Neat = None

    def __init__(self,neat:Neat):
        self.neat = neat
        self.connections = RandomHashSet()
        self.nodes = RandomHashSet()

    @staticmethod
    def empty_genome(neat:Neat):
        # return Genome
        g:Genome = Genome(neat)
        for i in range(neat.input_size+neat.output_size):
            g.nodes.add(neat.getNode(i+1))

        return g


    """
    calculated the distance between this genome g1 and a second genome g2
        - g1 must have the highest innovation number!
    """
    def distance(self, g2)->float:
        # g2 -> Genome
        g1 = self

        highest_innovation_gene1 = 0
        if g1.connections.size() != 0:
            highest_innovation_gene1 = g1.connections.get(g1.connections.size()-1).innovation_number

        highest_innovation_gene2 = 0
        if g2.connections.size() != 0:
            highest_innovation_gene2 = g2.connections.get(g2.connections.size()-1).innovation_number

        if highest_innovation_gene1 < highest_innovation_gene2:
            g = g1
            g1 = g2
            g2 = g


        index_g1:int = 0
        index_g2:int = 0
        disjoint:int = 0
        excess:int = 0
        weight_diff:float = 0
        similar:int = 0

        while index_g1 < g1.connections.size() and index_g2 < g2.connections.size():
            gene1:ConnectionGene = g1.connections.get(index_g1)
            gene2:ConnectionGene = g2.connections.get(index_g2)

            in1:int = gene1.innovation_number
            in2:int = gene2.innovation_number

            if in1 == in2:
                # similar gene
                similar += 1
                weight_diff += abs(gene1.weight-gene2.weight)
                index_g1 += 1
                index_g2 += 1
            elif in1 > in2:
                # disjoint gene of b
                disjoint += 1
                index_g2 += 1
            else:
                # disjoint gene of a
                disjoint += 1
                index_g1 += 1

        weight_diff /= max(1,similar)
        excess = g1.connections.size() - index_g1

        N = max(g1.connections.size(),g2.connections.size())
        if N < 20:
            N = 1

        return self.neat.C1 * disjoint/N + self.neat.C2 * excess / N + self.neat.C3 * weight_diff



    """
    * creates a new genome.
     * g1 should have the higher score
     *  - take all the genes of a
     *  - if there is a genome in a that is also in b, choose randomly
     *  - do not take disjoint genes of b
     *  - take excess genes of a if they exist
     return Genome
    """
    @staticmethod
    def crossover(g1, g2):
        neat:Neat = g1.neat

        genome = Genome.empty_genome(neat)

        index_g1 = 0
        index_g2 = 0

        while index_g1 < g1.connections.size() and index_g2 < g2.connections.size():

            gene1:ConnectionGene = g1.connections.get(index_g1)
            gene2:ConnectionGene = g2.connections.get(index_g2)

            in1:int = gene1.innovation_number
            in2:int = gene2.innovation_number

            if in1 == in2:
                # similar gene
                if random.random() > 0.5:
                    genome.connections.add(neat.connections[gene1])
                else:
                    genome.connections.add(neat.connections[gene2])
                index_g1 += 1
                index_g2 += 1
            elif in1 > in2:
                # disjoint gene of b
                # genome.connections.add(neat.connections[gene2])
                index_g2 += 1
            else:
                # disjoint gene of a
                genome.connections.add(neat.connections[gene1])
                index_g1 += 1

        
        while index_g1 < g1.connections.size():
            gene1:ConnectionGene = g1.connections.get(index_g1)
            genome.connections.add(neat.connections[gene1])
            index_g1 += 1

        for c in genome.connections.data:
            genome.nodes.add(c.from_gene) 
            genome.nodes.add(c.to_gene) 

        return genome

    def mutate(self):
        if random.random() < self.neat.PROBABILITY_MUTATE_LINK:
            self.mutateLink()
        if random.random() < self.neat.PROBABILITY_MUTATE_NODE:
            self.mutateNode()
        if random.random() < self.neat.PROBABILITY_MUTATE_WEIGHT_SHIFT:
            self.mutateWeightShift()
        if random.random() < self.neat.PROBABILITY_MUTATE_WEIGHT_RANDOM:
            self.mutateWeightRandom()
        if random.random() < self.neat.PROBABILITY_MUTATE_TOGGLE_LINK:
            self.mutateLinkToggle()

    
    def mutateLink(self):

        for i in range(100):
            a:NodeGene = self.nodes.randomElement()
            b:NodeGene = self.nodes.randomElement()

            if a is None or b is None:
                continue

            con:ConnectionGene = None
            if a.x < b.x:
                con = ConnectionGene(a,b)
            else:
                con = ConnectionGene(b,a)

            if self.connections.contains(con):
                continue

            con = self.neat.getConnection(con.from_gene, con.to_gene)
            con.weight = (random.random() * 2 - 1) * self.neat.WEIGHT_RANDOM_STRENGTH

            self.connections.addSorted(con)
            return

    def mutateNode(self):

        con:ConnectionGene = self.connections.randomElement()
        if con is None:
            return

        from_gene:NodeGene = con.from_gene
        to_gene:NodeGene = con.to_gene

        replace_index:int = self.neat.getReplaceIndex(from_gene,to_gene)

        middle:NodeGene = None

        if replace_index == 0:
            middle:NodeGene = self.neat.getNode()
            middle.x = (from_gene.x + to_gene.x)/2
            middle.y = (from_gene.y + to_gene.y)/2 + (random.random() * 0.1 - 0.05)
            self.neat.setReplaceIndex(from_gene,to_gene,middle.innovation_number)
        else:
            middle = self.neat.getNode(replace_index)

        

        con1:ConnectionGene = self.neat.getConnection(from_gene,middle)
        con2:ConnectionGene = self.neat.getConnection(middle,to_gene)

        con1.weight = 1
        con2.weight = con.weight
        con2.enabled = con.enabled

        self.connections.remove(con)
        self.connections.add(con1)
        self.connections.add(con2)

        self.nodes.add(middle)

    def mutateWeightShift(self): 
        con:ConnectionGene = self.connections.randomElement()
        if not con is None:
            con.weight = con.weight + ((random.random()*2-1) * self.neat.WEIGHT_SHIFT_STRENGTH)
        
    def mutateWeightRandom(self): 
        con:ConnectionGene = self.connections.randomElement()
        if not con is None:
            con.weight = ((random.random()*2-1) * self.neat.WEIGHT_RANDOM_STRENGTH)
        
    def mutateLinkToggle(self):
        con:ConnectionGene = self.connections.randomElement()
        if not con is None:
            con.enabled = not con.enabled
        

        











